# Requierments


In [1]:
!pip install findspark
!pip install confluent-kafka
# Downloaded from https://mvnrepository.com/artifact/org.apache.spark/spark-streaming-kafka-0-8-assembly_2.11
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar

--2020-04-13 19:11:08--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.112.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.112.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13385346 (13M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar.2’

spark-streaming-kaf 100%[===================>]  12.76M  3.73MB/s    in 3.6s    

2020-04-13 19:11:12 (3.56 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar.2’ saved [13385346/13385346]



In [2]:
import os
import findspark
findspark.init('/usr/local/spark/spark-2.4.0-bin-hadoop2.7')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar pyspark-shell'

---

In [3]:
from pyspark import SparkConf, SparkContext
# from operator import add
# import sys
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import pickle
from pyspark.mllib.feature import HashingTF, IDF, StandardScaler
from pyspark.mllib.classification import LogisticRegressionModel
from pyspark.mllib.linalg import Vectors

APP_NAME = "BigData"
sc = SparkContext(appName=APP_NAME)
ssc = StreamingContext(sc, 2)
sc

<SparkContext master=local[*] appName=BigData>

In [4]:
with open('./data/hashedTF.pickle', 'rb') as f:
    hashingTF = pickle.load(f)
    
htfVectors = sc.textFile("./data/htfVectors").map(Vectors.parse)
idf = IDF().fit(htfVectors)

tfidf =  idf.transform(htfVectors)

scaler = StandardScaler().fit(tfidf)

model = LogisticRegressionModel.load(sc, "./models/Logistic_Regression")

In [5]:
topic = "test"
brokers = "localhost:9092"
kvs = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": brokers})


In [6]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = []
    for i in range(0,len(payload)-n+1):
        ngrams.append(payload[i:i+n])
    return ngrams


def get_prediction(line):
    try:
        predictions = model.predict(scaler.transform(idf.transform(hashingTF.transform(line))))
        print(predictions.collect())
    except : 
        print('---------------No data----------------')



In [7]:
lines = ssc.socketTextStream("localhost", 9092)
ngrams = lines.flatMap(to_ngram)
ngrams.foreachRDD(get_prediction)

ssc.start()
ssc.awaitTermination()  

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 

In [ ]:
KafkaUtils.createDirectStream?